In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint

# Import API key
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


## Generate Cities List

In [20]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=100)
lngs = np.random.uniform(low=-180.000, high=180.000, size=100)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


76

## Perform API Calls

In [21]:
# Save config information

# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"


In [38]:
# set up lists to hold reponse info
cities_found = []
cloudiness = []
country = []
date = []
humidity = []
lat = []
lon = []
max_temp = []
wind_speed = []

# Loop through the list of cities and perform a request for data on each
for city in cities:
    params = {
        'appid' : api_key,
        'q' : city,
        'units' : 'imperal'
    }

    response = requests.get(url, params=params).json()
    
    if response['cod'] == 200:
        cities_found.append(city)
        cloudiness.append(response['clouds']['all'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        humidity.append(response['main']['humidity'])
        lat.append(response['coord']['lat'])
        lon.append(response['coord']['lon'])
        max_temp.append(response['main']['temp_max'])
        wind_speed.append(response['wind']['speed'])


In [37]:
pprint(response)

{'base': 'stations',
 'clouds': {'all': 0},
 'cod': 200,
 'coord': {'lat': 54.35, 'lon': 18.65},
 'dt': 1534989600,
 'id': 3099434,
 'main': {'humidity': 67,
          'pressure': 1017,
          'temp': 289.15,
          'temp_max': 289.15,
          'temp_min': 289.15},
 'name': 'Gdansk',
 'sys': {'country': 'PL',
         'id': 5349,
         'message': 0.004,
         'sunrise': 1534995386,
         'sunset': 1535047103,
         'type': 1},
 'visibility': 10000,
 'weather': [{'description': 'clear sky',
              'icon': '01n',
              'id': 800,
              'main': 'Clear'}],
 'wind': {'deg': 170, 'speed': 1.5}}


In [40]:
# create a data frame from cities, lat, and temp
weather_dict = {
    "City": cities_found,
    "Cloudiness": cloudiness,
    "Country": country,
    "Date": date,
    "Humidity": humidity,
    "Lat": lat,
    "Lon": lon,
    "Max Temp": max_temp,
    "Wind Speed": wind_speed,
}
weather_data = pd.DataFrame(weather_dict)
weather_data.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,kaitangata,92,NZ,1534993445,99,-46.28,169.85,280.77,3.02
1,boyolangu,0,ID,1534993682,55,-8.09,111.90,301.72,2.87
2,albany,75,US,1534992660,68,42.65,-73.75,294.25,4.60
3,qaanaaq,92,GL,1534993683,100,77.48,-69.36,275.37,2.07
4,puerto ayora,0,EC,1534993683,100,-0.74,-90.35,295.27,6.07
